# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA (Low-Rank Adaptation)
* Model: GPT-2 (via Hugging Face – 'gpt2')
* Evaluation approach: Accuracy comparison before and after fine-tuning using Hugging Face `Trainer`
* Fine-tuning dataset: 'ag_news' dataset from Hugging Face (for news classification, simple and fast) 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Load dataset
dataset = load_dataset("ag_news")
dataset = dataset.map(lambda x: {"labels": x["label"]}, remove_columns=["label"])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


# Tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Load model for classification
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=4)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# Define metric
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Training arguments
args = TrainingArguments(
    output_dir="/tmp/base_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    logging_dir="/tmp/logs",
    logging_steps=10,
    save_total_limit=1
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"].select(range(2000)),
    eval_dataset=tokenized_ds["test"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate base model
original_results = trainer.evaluate()
print("Original model performance:", original_results)


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Original model performance: {'eval_loss': 5.43803596496582, 'eval_accuracy': 0.294, 'eval_runtime': 4.3991, 'eval_samples_per_second': 113.66, 'eval_steps_per_second': 28.415}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [2]:
from peft import LoraConfig, get_peft_model

# Create LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

# Wrap base model in PEFT
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

# Training arguments for PEFT
peft_args = TrainingArguments(
    output_dir="/tmp/peft_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/tmp/peft_logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# PEFT Trainer
peft_trainer = Trainer(
    model=peft_model,
    args=peft_args,
    train_dataset=tokenized_ds["train"].select(range(2000)),
    eval_dataset=tokenized_ds["test"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train PEFT model
peft_trainer.train()

# Save PEFT adapter only
peft_model.save_pretrained("/tmp/peft_model")
tokenizer.save_pretrained("/tmp/peft_model")
# Re-save to a directory next to the notebook
peft_model.save_pretrained("./peft_model")
tokenizer.save_pretrained("./peft_model")


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 817,152 || all params: 125,256,960 || trainable%: 0.6523805144241086


Epoch,Training Loss,Validation Loss,Accuracy
1,1.003900,0.907947,0.678000
2,0.537200,0.614833,0.792000


('./peft_model/tokenizer_config.json',
 './peft_model/special_tokens_map.json',
 './peft_model/vocab.json',
 './peft_model/merges.txt',
 './peft_model/added_tokens.json',
 './peft_model/tokenizer.json')

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [7]:
# Saving the model
# model.save("/tmp/peft-model") - this is not working, already saved in previous step - check saved
import os
# Re-save to a directory next to the notebook
peft_model.save_pretrained("./peft_model")
tokenizer.save_pretrained("./peft_model")

print(os.listdir("./peft_model"))


['adapter_model.bin', 'adapter_config.json', 'README.md', 'merges.txt', 'tokenizer.json', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.json']


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from peft import PeftModel, PeftConfig

# 1. Load PEFT config
peft_config = PeftConfig.from_pretrained("./peft_model")

# 2. Load tokenizer (with pad token fix)
tokenizer = AutoTokenizer.from_pretrained("./peft_model")
tokenizer.pad_token = tokenizer.eos_token

# 3. Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=4
)
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.resize_token_embeddings(len(tokenizer))

# 4. Load adapter weights
peft_loaded = PeftModel.from_pretrained(base_model, "./peft_model")
peft_loaded.eval()

# 5. Re-evaluate using Trainer
peft_eval_trainer = Trainer(
    model=peft_loaded,
    args=args,
    eval_dataset=tokenized_ds["test"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 6. Evaluate and compare
peft_results = peft_eval_trainer.evaluate()
print("Fine-tuned PEFT model performance:", peft_results)

# 7. Compare with original
print("\n Comparison:")
print("Original:", original_results)
print("PEFT Fine-tuned:", peft_results)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Fine-tuned PEFT model performance: {'eval_loss': 0.6148329973220825, 'eval_accuracy': 0.792, 'eval_runtime': 4.5294, 'eval_samples_per_second': 110.389, 'eval_steps_per_second': 27.597}

 Comparison:
Original: {'eval_loss': 5.43803596496582, 'eval_accuracy': 0.294, 'eval_runtime': 4.3991, 'eval_samples_per_second': 113.66, 'eval_steps_per_second': 28.415}
PEFT Fine-tuned: {'eval_loss': 0.6148329973220825, 'eval_accuracy': 0.792, 'eval_runtime': 4.5294, 'eval_samples_per_second': 110.389, 'eval_steps_per_second': 27.597}


In [5]:

# 1. Try using the bitsandbytes package (installed in the workspace) to combine quantization and LoRA. This is also known as QLoRA

from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig
import torch


# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load quantized model (4-bit GPT-2 with correct num_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=4,
    quantization_config=bnb_config,
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

# Merge quantization + LoRA = QLoRA
qlora_model = get_peft_model(model, lora_config)
qlora_model.print_trainable_parameters()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 817,152 || all params: 125,256,960 || trainable%: 0.6523805144241086


In [6]:
# 2. Try training the model using different PEFT configurations and compare the results

configs_to_try = [
    {"r": 4, "alpha": 8},
    {"r": 8, "alpha": 16},
    {"r": 16, "alpha": 32}
]

for cfg in configs_to_try:
    lora_config = LoraConfig(
        r=cfg["r"],
        lora_alpha=cfg["alpha"],
        target_modules=["c_attn", "c_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type="SEQ_CLS"
    )
    
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_ds["train"].select(range(2000)),
        eval_dataset=tokenized_ds["test"].select(range(500)),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    results = trainer.evaluate()
    print(f"Results for r={cfg['r']}, alpha={cfg['alpha']}: {results}")


trainable params: 405,504 || all params: 124,851,456 || trainable%: 0.3247891638524424


Epoch,Training Loss,Validation Loss,Accuracy
1,1.554900,1.734725,0.288000


Results for r=4, alpha=8: {'eval_loss': 1.7347248792648315, 'eval_accuracy': 0.288, 'eval_runtime': 4.7637, 'eval_samples_per_second': 104.961, 'eval_steps_per_second': 26.24, 'epoch': 1.0}
trainable params: 811,008 || all params: 125,256,960 || trainable%: 0.6474753977743033


Epoch,Training Loss,Validation Loss,Accuracy
1,1.406900,1.557835,0.336000


Checkpoint destination directory /tmp/base_model/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Results for r=8, alpha=16: {'eval_loss': 1.5578352212905884, 'eval_accuracy': 0.336, 'eval_runtime': 4.7283, 'eval_samples_per_second': 105.746, 'eval_steps_per_second': 26.437, 'epoch': 1.0}
trainable params: 1,622,016 || all params: 126,067,968 || trainable%: 1.2866202459930187


Epoch,Training Loss,Validation Loss,Accuracy
1,1.248900,1.376761,0.410000


Checkpoint destination directory /tmp/base_model/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Results for r=16, alpha=32: {'eval_loss': 1.3767609596252441, 'eval_accuracy': 0.41, 'eval_runtime': 4.8188, 'eval_samples_per_second': 103.76, 'eval_steps_per_second': 25.94, 'epoch': 1.0}


In [8]:
all_results = []

for cfg in configs_to_try:
    ...
    result = trainer.evaluate()
    all_results.append((cfg["r"], cfg["alpha"], result["eval_accuracy"]))

print("\nSummary of LoRA Config Results:")
for r, alpha, acc in all_results:
    print(f"r={r}, alpha={alpha} => Accuracy: {acc:.3f}")



Summary of LoRA Config Results:
r=4, alpha=8 => Accuracy: 0.410
r=8, alpha=16 => Accuracy: 0.410
r=16, alpha=32 => Accuracy: 0.410
